<a href="https://colab.research.google.com/github/guilhermelaviola/IntelligentCommunication/blob/main/Class07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing all the necessary libraries and resources:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Defining the classes Question and Answer:
class Question:
    def __init__(self, id, text):
        self.id = id
        self.text = text

class Answer:
    def __init__(self, id, text, question_id):
        self.id = id
        self.text = text
        self.question_id = question_id

# Defining the class to manipulate the database:
class QADatabase:
    def __init__(self):
        self.Questions = []
        self.Answers = []

    def add_question(self, Question):
        self.Questions.append(Question)

    def add_answer(self, Answer):
        self.Answers.append(Answer)

    def find_answer(self, Question_text):
        # Checking if there are questions and answers:
        if not self.Questions or not self.Answers:
            return None

        # Creating a new list with texts of questions and answers:
        texts = [p.text for p in self.Questions] + [r.text for r in self.Answers]

        # Using TF-IDF to represent texts:
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(texts)

        # Extracting the vector of the user question:
        question_vector = vectorizer.transform([question_text])
        answers_vector = tfidf_matrix[len(self.Questions):]  # Vetores de Answers

        # Calculating the similarity:
        similarities = cosine_similarity(question_vector, answers_vector).flatten()

        # Checking the best correspondence:
        best_correspondence = similarities.argmax()
        if similarities[best_correspondence] > 0:
            return self.Answers[best_correspondence]
        return None


# Creating the questions and answers database:
db = QADatabase()

# Adding the questions:
db.add_question(Question(1, "What's the capital city of Italy?"))
db.add_question(Question(2, "Who was Queen's frontman?"))

# Adding the answers:
db.add_answer(Answer(1, "The capital city of Italy is Rome.", 1))
db.add_answer(Answer(2, "Freddie Mercury.", 2))

# Loop for the user to make questions:
while True:
    user_question = input("\nEnter your question (or type 'exit' to leave): ")
    if user_question.lower() == 'exit':
        print("Ending the program.")
        break

    answer_found = db.find_answer(user_question)

    if answer_found:
        print(f"Answer found: {answer_found.text}")
    else:
        print("Answer not found.")
